In [23]:
import cobra
import pandas as pd
import cobra_fba as cfba

In [119]:
model = cfba.FBA({'model_path': '../../models/json/iML1515.json'})

In [120]:
phaA = cobra.Reaction('PHAA')
phaA.name = 'Acetyl-CoA C-acetyltransferase'
phaA.subsystem = 'pha Plasmid'
phaA.lower_bound = -1000.
phaA.upper_bound = 1000.

phaB = cobra.Reaction('PHAB')
phaB.name = 'Acetoacetyl-CoA Reductase'
phaB.subsystem = 'pha Plasmid'
phaB.lower_bound = -1000.
phaB.upper_bound = 1000.

phaC = cobra.Reaction('PHAC')
phaC.name = 'PHA Synthase Subunit PhaC'
phaC.subsystem = 'pha Plasmid'
phaC.lower_bound = -1000.
phaC.upper_bound = 1000.

In [121]:
hbcoa__3R_c = cobra.Metabolite(
    'hbcoa__3R_c',
    formula = 'C25H38N7O18P3S',
    name = '(R)-3-Hydroxybutanoyl-CoA',
    compartment = 'c')

phb_c = cobra.Metabolite(
    'phb_c',
    formula = 'C4H6O2',
    name = '(R)-3-hydroxybutyrate',
    compartment = 'c')

accoa_c = model.model.metabolites.accoa_c

aacoa_c = model.model.metabolites.aacoa_c

coa_c = model.model.metabolites.coa_c

nadph_c = model.model.metabolites.nadph_c

nadp_c = model.model.metabolites.nadp_c

h_c = model.model.metabolites.h_c


In [122]:
phaA.add_metabolites({
    accoa_c: -2.0,
    aacoa_c: 1.0,
    coa_c: 1.0
})

phaB.add_metabolites({
    aacoa_c: -1.0,
    h_c: -1.0,
    nadph_c: -1.0,
    nadp_c: 1.0,
    hbcoa__3R_c: 1.0
})

phaC.add_metabolites({
    hbcoa__3R_c: -1.0,
    phb_c: 1.0,
    coa_c: 1.0
})

In [123]:
phaA.gene_reaction_rule = '( phaA )'
phaB.gene_reaction_rule = '( phaB )'
phaC.gene_reaction_rule = '( phaC )'

In [124]:
model.model.add_reactions([phaA, phaB, phaC])
# model.model.add_boundary(phb_c, type = 'exchange')

In [125]:
model.model.reactions.PHAA

Reaction identifier,PHAA
Name,Acetyl-CoA C-acetyltransferase
Memory address,0x07fed04370640
Stoichiometry,2.0 accoa_c <=> aacoa_c + coa_c 2.0 Acetyl-CoA <=> Acetoacetyl-CoA + Coenzyme A
GPR,( phaA )
Lower bound,-1000.0
Upper bound,1000.0


In [126]:
model.model.reactions.PHAB

Reaction identifier,PHAB
Name,Acetoacetyl-CoA Reductase
Memory address,0x07fed04370580
Stoichiometry,aacoa_c + h_c + nadph_c <=> hbcoa__3R_c + nadp_c Acetoacetyl-CoA + H+ + Nicotinamide adenine dinucleotide phosphate - reduced <=> (R)-3-Hydroxybutanoyl-CoA + Nicotinamide adenine dinucleotide phosphate
GPR,( phaB )
Lower bound,-1000.0
Upper bound,1000.0


In [127]:
model.model.reactions.PHAC

Reaction identifier,PHAC
Name,PHA Synthase Subunit PhaC
Memory address,0x07fed04370670
Stoichiometry,hbcoa__3R_c <=> coa_c + phb_c (R)-3-Hydroxybutanoyl-CoA <=> Coenzyme A + (R)-3-hydroxybutyrate
GPR,( phaC )
Lower bound,-1000.0
Upper bound,1000.0


In [113]:
cobra.io.save_json_model(model.model, '../../models/json/test.json')

In [128]:
model.model.objective = 'PHAB'
print(model.model.objective.expression)
print(model.model.objective.direction)


1.0*PHAB - 1.0*PHAB_reverse_34919
max


In [129]:
model.model.optimize()
print(model.model.metabolites.accoa_c.summary())
print(model.model.metabolites.hbcoa__3R_c.summary())
print(model.model.metabolites.nadph_c.summary())

accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent   Flux Reaction                                         Definition
100.00% 0.5838      PDH coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c

Consuming Reactions
-------------------
Percent    Flux Reaction                                      Definition
100.00% -0.5838       CS accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c
hbcoa__3R_c
Formula: C25H38N7O18P3S

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []
nadph_c
Formula: C21H26N7O17P3

Producing Reactions
-------------------
Percent   Flux Reaction                                  Definition
100.00% 0.5838   ICDHyr icit_c + nadp_c <=> akg_c + co2_c + nadph_c

Consuming Reactions
-------------------
Percent    Flux Reaction                          Definition
100.00% -0.5838 

In [63]:
model.minimal_external()

{'glc__D_e': 0.623636363636364, 'o2_e': 1.2472727272727284}

In [84]:
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model, validate_sbml_model
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    write_sbml_model(model.model, filename=f_sbml.name)
    report = validate_sbml_model(filename=f_sbml.name)

pprint(report)

(<Model iML1515 at 0x7feceb8c2940>,
 {'COBRA_CHECK': [],
  'COBRA_ERROR': [],
  'COBRA_FATAL': [],
  'COBRA_WARNING': [],
  'SBML_ERROR': [],
  'SBML_FATAL': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': []})


In [102]:
print(dir(model.model.metabolites.get_by_name('(S)-3-Hydroxybutanoyl-CoA')))

AttributeError: DictList has no attribute or entry get_by_name